# Reinforcement learning

Goals:
- hand-code the value iteration algorithm on a toy dataset
- find the corresponding policy

## Import basic libraries

In [1]:
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

## Create a toy dataset

In [2]:
# 4x4 state space S
N = 4
M = 4

# Reward function (rewards given for getting in each of the states)
R = np.array([[-1, -1, -1, -1],
                  [-1, -1, -1, 100],
                  [-1, -1, -100, -1],
                  [-1, -1, -1, -1]])

# action space A is { MOVE_LEFT, MOVE_RIGHT, MOVE_UP, MOVE_DOWN, STAY }
LEFT = 0
RIGHT = 1
UP = 2
DOWN = 3
STAY = 4

# when the agent decides to move in any direction, 
# it will actually move in that direction with 0.8 probability...
p_direct = 0.8
# ...and diverge to the "lateral" state with 0.1 probability
p_diverge = 0.1
# if there is a wall in a direction of the movement, 
# the agent will simply bounce of the wall and stay where it is

# discount factor
gamma = 0.90

In [3]:
# start position
start = (3, 0)

## Value iteration

In [4]:
def bounce_if_hit_wall(new_pos, old_pos):
    x, y = old_pos
    x1, y1 = new_pos
    
    if x1 < 0 or x1 >= M or y1 < 0 or y1 >= N:
        x1, y1 = x, y
    
    return x1, y1
            

def get_next_positions(action, pos):
    x, y = pos

    if action == LEFT:
        next_pos_direct = x - 1, y
        next_pos_diverge1 = x, y - 1
        next_pos_diverge2 = x, y + 1
    elif action == RIGHT:
        next_pos_direct = x + 1, y
        next_pos_diverge1 = x, y - 1
        next_pos_diverge2 = x, y + 1
    elif action == UP:
        next_pos_direct = x, y - 1
        next_pos_diverge1 = x - 1, y
        next_pos_diverge2 = x + 1, y
    elif action == DOWN:
        next_pos_direct = x, y + 1
        next_pos_diverge1 = x - 1, y
        next_pos_diverge2 = x + 1, y
        
    next_pos_direct = bounce_if_hit_wall(next_pos_direct, pos)
    next_pos_diverge1 = bounce_if_hit_wall(next_pos_diverge1, pos)
    next_pos_diverge2 = bounce_if_hit_wall(next_pos_diverge2, pos)
    
    return next_pos_direct, next_pos_diverge1, next_pos_diverge2

In [5]:
def get_next_action_value(action, pos, V):
    next_pos_direct, next_pos_diverge1, next_pos_diverge2 = get_next_positions(action, pos)
    
    x, y = next_pos_direct
    x1, y1 = next_pos_diverge1
    x2, y2 = next_pos_diverge2
    
    return gamma * (V[y, x] * p_direct + V[y1, x1] * p_diverge + V[y2, x2] * p_diverge)

In [6]:
def value_iteration(V):
    V_next = np.copy(V)
    for y in range(N):
        for x in range (M):
            # negative states are sticky
            if R[y, x] == -100:
                V_next[y, x] = R[y, x] + gamma * V[y, x]
            else:
                value_stay = gamma * V[y, x]
                value_left = get_next_action_value(LEFT, (x, y), V)
                value_right = get_next_action_value(RIGHT, (x, y), V)
                value_up = get_next_action_value(UP, (x, y), V)
                value_down = get_next_action_value(DOWN, (x, y), V)
                
                V_next[y, x] = R[y, x] + max(value_stay, value_left, value_right, value_up, value_down)
    return V_next

In [7]:
V = np.zeros((N, M))
cnt = 0

V_prev = value_iteration(V)
while(True):
    V = value_iteration(V_prev)
    cnt = cnt + 1
    # if cnt == 1000: break;
    if np.array_equal(V, V_prev): break;
    V_prev = V
    
print("Iterations: " + str(cnt))
V

Iterations: 338


array([[  566.71576199,   652.67442452,   751.77047051,   864.46081576],
       [  519.8417529 ,   596.21097272,   696.65934235,  1000.        ],
       [  450.72704408,   409.72831133, -1000.        ,   691.20879121],
       [  391.28497119,   361.62057835,   360.00607541,   581.39656754]])

In [8]:
MIN_INF = -1000000

def get_policy(V):
    P = np.zeros((N, M), dtype='int16')
    for y in range(N):
        for x in range (M):
            # negative states are sticky
            if R[y, x] == -100:
                P[y, x] = STAY
            else:
                if x > 0: left = V[y, x - 1]
                else: left = MIN_INF

                if x < M - 1: right = V[y, x + 1]
                else: right = MIN_INF

                if y > 0: up = V[y - 1, x]
                else: up = MIN_INF

                if y < N - 1: down = V[y + 1, x]
                else: down = MIN_INF

                stay = V[y, x]

                action = np.argmax(np.array([left, right, up, down, stay]))
                P[y, x] = action
    return P

In [9]:
P = get_policy(V)

In [10]:
def to_action(P):
    if P == LEFT: return '←'
    elif P == RIGHT: return '→'
    elif P == UP: return '↑'
    elif P == DOWN: return '↓'
    elif P == STAY: return '↻'

to_action = np.vectorize(to_action)
to_action(P)

array([['→', '→', '→', '↓'],
       ['→', '→', '→', '↻'],
       ['↑', '↑', '↻', '↑'],
       ['↑', '↑', '→', '↑']], dtype='<U1')

## Conclusions

- The value iteration algorithm converges after 338 iterations
- The resulting policy mostly makes sense
- One thing I don't get is why the algorithm is not taking the safest route ever, meaning walking along the upper boundary